In [ ]:
# inspiration: https://docs.google.com/spreadsheets/d/1wZhPLMCHKJvwOkP4juclhjFgqIY8fQFMemwKL2c64vk/htmlview row 1252
# senate data: https://senatestockwatcher.com/api
# house data: https://housestockwatcher.com/api
# historical/current legislator data: https://github.com/unitedstates/congress-legislators

In [1]:
import pandas as pd

## Process legislator file

In [2]:
# read in current & historical congressional member lists
cols = ["first_name","middle_name", "last_name", "full_name", "type", "party"]
congress_historical = pd.read_csv("https://theunitedstates.io/congress-legislators/legislators-historical.csv", usecols=cols)
congress_current    = pd.read_csv("https://theunitedstates.io/congress-legislators/legislators-current.csv",    usecols=cols)

# narrow to senators (full_name <> na seems to weed out most old senators)
congress_historical = congress_historical[(congress_historical.type == "sen") & (congress_historical.full_name.notna())]
congress_current    = congress_current[(congress_current.type == "sen") & (congress_current.full_name.notna())]

# concat them
congress = pd.concat([congress_historical, congress_current])
congress.head()

,last_name,first_name,middle_name,full_name,type,party
11634,Inouye,Daniel,K.,Daniel K. Inouye,sen,Democrat
11635,Akaka,Daniel,Kahikina,Daniel K. Akaka,sen,Democrat
11636,Bingaman,Jeff,NaN,Jeff Bingaman,sen,Democrat
11637,Conrad,Kent,Kent,Kent Conrad,sen,Democrat
11638,Hutchison,Kay,Bailey,Kay Bailey Hutchison,sen,Republican


In [3]:
congress.to_csv("data/senators.csv", index=False)

## Get daily senate trade summary file 

In [4]:
# url = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_daily_summaries.json"
txns = pd.read_json("https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json")

# narrow columns
txns = txns[["senator", "transaction_date", "disclosure_date", "ticker", "asset_description", "asset_type", "amount"]]

print(txns.shape)
txns.head(1)

(8831, 7)


,senator,transaction_date,disclosure_date,ticker,asset_description,asset_type,amount
0,Sheldon Whitehouse,08/30/2021,09/13/2021,BABA,Alibaba Group Holding Limited American Depositary,Stock,"$1,001 - $15,000"


In [5]:
txns.to_csv("data/txns_09132021.csv", index=False)

## Join congress to txns
First, standardize names for a successful merge

In [6]:
congress_to_replace = {", IV": "", ", III": "", "Tommy": "Thomas", ", Jr.": "", "Bill Hagerty": "William Hagerty", 
                       "Mike Crapo": "Michael Crapo", "Joe Manchin": "Joseph Manchin", "Rob Portman": "Robert Portman", 
                       "Thom Tillis": "Thomas Tillis", "John Reed": "Jack Reed", "Tim Kaine": "Timothy Kaine", 
                       "Dan Sullivan": "Daniel Sullivan", "Bill Cassidy": "William Cassidy", "Jacky Rosen": "Jacklyn Rosen"}
              
congress["name_cleaned"] = congress["full_name"].replace(congress_to_replace, regex=True)

In [7]:
def clean_congress_name(fullname):
    names = fullname.split(" ")
    fullname = names[0] + " " + names[-1]
    return fullname

congress["fullname_cleaned"] = congress["name_cleaned"].apply(clean_congress_name)

In [8]:
txn_to_replace = {", Jr.": "", ", Iv" : "", "Ladda ": "", "A. ": "", " , Jr": "", "John F Reed": "Jack Reed",
                  "Rafael": "Ted", ", Iii": "", "Moran,": "Moran", "Ronald L Wyden": "Ron Wyden",
                  "Mitchell Mcconnell": "Mitch McConnell", "Thomas Udall": "Tom Udall"}

txns["name_cleaned"] = txns["senator"].replace(txn_to_replace, regex=True)

In [9]:
def get_first_last_name(row):
    fullname = row["name_cleaned"]
    names = fullname.split(" ")
    first_name = names[0]
    last_name = names[-1]
    full_name = f"{first_name} {last_name}"
    return full_name, first_name, last_name

txns[["full_name", "first_name", "last_name"]] = txns.apply(get_first_last_name, axis=1, result_type="expand")

merge

In [10]:
merged = txns.merge(congress,
                    left_on="full_name",
                    right_on="fullname_cleaned",
                    how="left")
merged.head()

,senator,transaction_date,disclosure_date,ticker,asset_description,asset_type,amount,name_cleaned_x,full_name_x,first_name_x,last_name_x,last_name_y,first_name_y,middle_name,full_name_y,type,party,name_cleaned_y,fullname_cleaned
0,Sheldon Whitehouse,08/30/2021,09/13/2021,BABA,Alibaba Group Holding Limited American Depositary,Stock,"$1,001 - $15,000",Sheldon Whitehouse,Sheldon Whitehouse,Sheldon,Whitehouse,Whitehouse,Sheldon,NaN,Sheldon Whitehouse,sen,Democrat,Sheldon Whitehouse,Sheldon Whitehouse
1,Sheldon Whitehouse,08/27/2021,09/13/2021,--,"MACYS RETAIL HLDGS INC <div class=""text-muted""...",Corporate Bond,"$15,001 - $50,000",Sheldon Whitehouse,Sheldon Whitehouse,Sheldon,Whitehouse,Whitehouse,Sheldon,NaN,Sheldon Whitehouse,sen,Democrat,Sheldon Whitehouse,Sheldon Whitehouse
2,Sheldon Whitehouse,08/30/2021,09/13/2021,MCHP,Microchip Technology Incorporated - Common Stock,Stock,"$15,001 - $50,000",Sheldon Whitehouse,Sheldon Whitehouse,Sheldon,Whitehouse,Whitehouse,Sheldon,NaN,Sheldon Whitehouse,sen,Democrat,Sheldon Whitehouse,Sheldon Whitehouse
3,Sheldon Whitehouse,08/30/2021,09/13/2021,T,AT&amp;T Inc.,Stock,"$15,001 - $50,000",Sheldon Whitehouse,Sheldon Whitehouse,Sheldon,Whitehouse,Whitehouse,Sheldon,NaN,Sheldon Whitehouse,sen,Democrat,Sheldon Whitehouse,Sheldon Whitehouse
4,Patrick J Toomey,01/16/2021,09/09/2021,--,Fitso (Exchanged) <br> Zomato (Received) <div ...,Non-Public Stock,"$1,001 - $15,000",Patrick J Toomey,Patrick Toomey,Patrick,Toomey,Toomey,Patrick,J.,Patrick J. Toomey,sen,Republican,Patrick J. Toomey,Patrick Toomey


In [11]:
# check for leaks
merged[merged.fullname_cleaned.isna()].full_name_x.unique()

array([], dtype=object)

In [12]:
# remove some extraneous name cols
merged = merged.drop(columns=["first_name_x", "first_name_y", 
                              "last_name_x", "last_name_y",
                              "name_cleaned_x", "full_name_x"])

In [13]:
merged.to_csv("data/txns_with_party.csv", index=False)